In [2]:
import pandas as pd
import numpy as np

In [10]:
df = pd.read_csv('orders.csv - orders.csv')
df[:5]

,id,order_date,ship_mode,customer_id,sales
0,100006,2014-09-07,Standard,DK-13375,377.970
1,100090,2014-07-08,Standard,EB-13705,699.192
2,100293,2014-03-14,Standard,NF-18475,91.056
3,100328,2014-01-28,Standard,JC-15340,3.928
4,100363,2014-04-08,Standard,JM-15655,21.376


In [11]:
df.dtypes

id               int64
order_date      object
ship_mode       object
customer_id     object
sales          float64
dtype: object

In [12]:
# Преобразуем колонку даты.
df['order_date'] = pd.to_datetime(df['order_date'])

In [13]:
df.dtypes

id                      int64
order_date     datetime64[ns]
ship_mode              object
customer_id            object
sales                 float64
dtype: object

# считаем покупки и выручку

In [14]:
df['sales'].sum()

2297200.8603000003

In [15]:
df['sales'].count()

5009

### Теперь посчитаем обе метрики для каждого пользователя. Сгруппируем датафрейм по полю customer_id:

In [16]:
df.groupby('customer_id')['sales'].agg(['sum', 'count'])

,sum,count
customer_id,,
AA-10315,5563.560,5
AA-10375,1056.390,9
AA-10480,1790.512,4
AA-10645,5086.935,6
AB-10015,886.156,3
...,...,...
XP-21865,2374.658,11
YC-21895,5454.350,5
YS-21880,6720.444,8


In [17]:
df.columns

Index(['id', 'order_date', 'ship_mode', 'customer_id', 'sales'], dtype='object')

In [20]:
first_orders = df.groupby('customer_id')['order_date'].agg({'first_order': 'min'})
first_orders.head()

SpecificationError: nested renamer is not supported

In [31]:
# Находим дату первой покупки
first_orders = df.groupby('customer_id')['order_date'].agg(['min'])
first_orders.head()

,min
customer_id,
AA-10315,2014-03-31
AA-10375,2014-04-21
AA-10480,2014-05-04
AA-10645,2014-06-22
AB-10015,2014-02-18


## Строим когорты

In [33]:
# Итак, когортами будем считать людей, сделавших первую покупку в тот или иной день. 
# Метрики для отслеживания — количество заказов и выручка.
df_merged = df.merge(first_orders, how='inner', left_on = 'customer_id', right_index=True)
df_merged[:5]

,id,order_date,ship_mode,customer_id,sales,min
0,100006,2014-09-07,Standard,DK-13375,377.970,2014-09-07
1219,131884,2015-12-06,Same Day,DK-13375,594.002,2014-09-07
1398,145065,2015-12-12,First,DK-13375,32.308,2014-09-07
3463,133046,2017-07-27,Second,DK-13375,297.990,2014-09-07
4102,165099,2017-12-11,First,DK-13375,1.392,2014-09-07


In [34]:
# Агрегируем по дате первой покупки и посчитаем нужные показатели:
df_merged.groupby('min')['sales'].agg(['sum', 'count'])

,sum,count
min,,
2014-01-03,1050.636,9
2014-01-04,1056.858,5
2014-01-05,1428.231,7
2014-01-06,14287.557,22
2014-01-07,1055.981,4
...,...,...
2017-07-08,1058.108,1
2017-09-15,1038.260,1
2017-10-21,729.648,1


In [35]:
df_merged.groupby(['min', 'order_date'])['sales'].agg(['sum', 'count'])

sum  count
min        order_date                 
2014-01-03 2014-01-03    16.448      1
           2014-11-12   153.112      1
           2015-04-18   209.550      1
           2015-11-24   383.610      1
           2016-05-15     7.764      1
...                         ...    ...
2017-09-15 2017-09-15  1038.260      1
2017-10-21 2017-10-21   729.648      1
2017-10-23 2017-10-23   863.880      1
2017-11-05 2017-11-05   796.036      1
           2017-11-18   409.548      1

[4621 rows x 2 columns]

In [36]:
df_merged['order_date']-df_merged['min']

0         0 days
1219    455 days
1398    461 days
3463   1054 days
4102   1191 days
          ...   
4161      0 days
4397      0 days
4992    823 days
4477      0 days
5003    538 days
Length: 5009, dtype: timedelta64[ns]

In [37]:
df_merged['order_date']-df_merged['min'] <='365 days'

0        True
1219    False
1398    False
3463    False
4102    False
        ...  
4161     True
4397     True
4992    False
4477     True
5003    False
Length: 5009, dtype: bool

In [38]:
year_1_f = df_merged['order_date']-df_merged['min'] <='365 days'
year_1_o = df_merged[year_1_f]
year_1_o[:5]

,id,order_date,ship_mode,customer_id,sales,min
0,100006,2014-09-07,Standard,DK-13375,377.970,2014-09-07
1,100090,2014-07-08,Standard,EB-13705,699.192,2014-07-08
369,129938,2014-12-15,Second,EB-13705,445.802,2014-07-08
1164,128125,2015-03-31,Standard,EB-13705,120.756,2014-07-08
2,100293,2014-03-14,Standard,NF-18475,91.056,2014-03-14


In [40]:
cohorts = year_1_o.groupby(['min', 'order_date'])['sales'].agg(['sum', 'count'])
cohorts.head()

sum  count
min        order_date                 
2014-01-03 2014-01-03    16.448      1
           2014-11-12   153.112      1
2014-01-04 2014-01-04   288.060      1
2014-01-05 2014-01-05    19.536      1
2014-01-06 2014-01-06  4407.100      3

In [41]:
# Последний шаг: посчитаем, сколько в среднем заказов и приносят клиенты в течение первого года.
#     Для этого сначала просуммируем показатели каждой когорты, а затем усредним значения методом mean()
cohorts.groupby('min').sum().mean()

sum      1949.850803
count       3.988789
dtype: float64